In [1]:
%pylab qt5
warnings.filterwarnings("ignore")
import lib2
from lib2.fulaut.MeasurementRunner import *
from importlib import reload

Populating the interactive namespace from numpy and matplotlib


In [2]:
MR = MeasurementRunner("Xmon 1910-103-5", "S23", {'vna': ['vna4'], 'mw_src': ['psg2'], 'cur_src': ['yok6']})

The device vna4 is detected as ZNB
The device psg2 is detected as PSG
The device yok6 is detected as YOK1


In [ ]:
MR.run([4,5])

Done: 0.00%, 0/121, [1.605e-04, -1.046e-06, 5.398e+00, 6.174e-01, 1.191e-01]2e-01]

DEBUG:fulaut:Two-tone fit: [ 1.60520417e-04 -1.04551848e-06  5.39769508e+09  6.17357783e-01
  1.19127402e-01]


DEBUG:fulaut:Measurement V-AC-Stark init, devs: {'vna': [ZNB], 'mw_src': [<drivers.E8257D.EXG object at 0x0000023932242780>], 'current_src': [<drivers.Yokogawa_GS200.Yokogawa_GS210 object at 0x0000023932242C88>]}


Measurement result 'V-ac-stark' for the sample 'Xmon 1910-103-5' not found
Started at:  2020-02-20 11:57:39.137339
Time left: 0 h 0 m 0.0 s, [Readout power [dBm]: -3.50e+01, Frequency [Hz]: 5.45e+09], average cycle time: 0.17 s            
Elapsed time: 0 h 18 m 8.49 s


DEBUG:fulaut:Transmon bare frequency: 5.4343 GHz, readout power (on SA): -57 dBm
DEBUG:fulaut:Pulsed measurements at 5.434 GHz, -1.05e-06 A, 0.00e+00 periods away from sws
DEBUG:fulaut:Requesting readout mixer cal: frozenset({('if_frequency', 0.0), ('lo_power', -10), ('lo_frequency', 6880000000.0), ('waveform_resolution', 1), ('ssb_power', -57)})
DEBUG:fulaut:Measurement V-rabi init, devs: {'vna': [ZNB], 'q_lo': [<drivers.E8257D.EXG object at 0x0000023932242780>], 'q_awg': [<drivers.IQAWG.IQAWG object at 0x0000023932256668>], 'ro_awg': [<drivers.IQAWG.IQAWG object at 0x00000239322565F8>]}


Detecting a resonator within provided frequency range of the VNA (6879346735.532153, 6882346735.532153)                    
Detected frequency is 6.88104 GHz, at 0.14 mU and -103.04 degrees
Figure with window name 'Resonator fit' not found
Started at:  2020-02-20 12:15:57.758176
Time left: 0 h 0 m 0.0 s, [excitation_duration: 5.00e+02], average cycle time: 5.94 s          

DEBUG:fulaut:Measurement V-ramsey init, devs: {'vna': [ZNB], 'ro_awg': [<drivers.IQAWG.IQAWG object at 0x00000239322565F8>], 'q_awg': [<drivers.IQAWG.IQAWG object at 0x0000023932256668>], 'q_lo': [<drivers.E8257D.EXG object at 0x0000023932242780>]}



Elapsed time: 0 h 19 m 53.93 s
Detecting a resonator within provided frequency range of the VNA (6879346735.532153, 6882346735.532153)                    
Detected frequency is 6.88108 GHz, at 0.28 mU and 134.18 degrees
Figure with window name 'Resonator fit' not found
Started at:  2020-02-20 12:36:00.447911
Time left: 0 h 0 m 0.0 s, [ramsey_delay: 5.00e+02], average cycle time: 6.67 s          
Elapsed time: 0 h 22 m 20.58 s


DEBUG:fulaut:Measurement V-rabi init, devs: {'vna': [ZNB], 'q_lo': [<drivers.E8257D.EXG object at 0x0000023932242780>], 'q_awg': [<drivers.IQAWG.IQAWG object at 0x0000023932256668>], 'ro_awg': [<drivers.IQAWG.IQAWG object at 0x00000239322565F8>]}


Detecting a resonator within provided frequency range of the VNA (6879346735.532153, 6882346735.532153)                    
Detected frequency is 6.88106 GHz, at 0.21 mU and -106.14 degrees
Figure with window name 'Resonator fit' not found
Started at:  2020-02-20 12:58:30.798929
Time left: 0 h 4 m 1.99 s, [excitation_duration: 3.98e+02], average cycle time: 5.9 s          

In [ ]:
import lib2.DispersiveHahnEcho
reload(lib2.DispersiveHahnEcho)
from lib2.DispersiveHahnEcho import *

In [ ]:
MR._max_echo_delay = 100e3
MR._echo_nop = 21

In [ ]:
qubit_name= "V"
DHE = DispersiveHahnEcho("%s-echo" % qubit_name,
                              MR._sample_name,
                              vna=MR._vna,
                              ro_awg=[MR._ro_awg],
                              q_awg=[MR._q_awg],
                              q_lo=MR._mw_src)

vna_parameters = {"bandwidth": 10,
                  "freq_limits": MR._res_limits[qubit_name],
                  "nop": 50,
                  "averages": 1,
                  "res_find_nop": 401}

echo_delays = linspace(0, MR._max_echo_delay, MR._echo_nop)
exc_frequency = MR._exact_qubit_freqs[qubit_name]
pi_pulse_duration = \
    MR._dro_results[qubit_name].get_pi_pulse_duration() * 1e3

pulse_sequence_parameters = \
    {"awg_trigger_reaction_delay": 0,
     "readout_duration": 6e3,
     "repetition_period": MR._max_echo_delay + 100e3,
     "half_pi_pulse_duration": pi_pulse_duration / 2}

ro_awg_params = {"calibration": MR._ro_cal}
q_awg_params = {"calibration": MR._exc_cal}

mw_src_params = {'power': MR._exc_cal.get_radiation_parameters()["lo_power"],
                 'frequency': exc_frequency + MR._exc_cal.get_radiation_parameters()["if_frequency"]}

DHE.set_fixed_parameters(pulse_sequence_parameters,
                        vna=[vna_parameters],
                        ro_awg=[ro_awg_params],
                        q_awg=[q_awg_params],
                        q_lo=[mw_src_params])
DHE.set_swept_parameters(echo_delays)
MeasurementResult.close_figure_by_window_name("Resonator fit")

dhe_result = DHE.launch()

In [ ]:
dhe_result.save()

In [ ]:
MR._dro_results["V"]

In [ ]:
print(MR._ro_cal)

In [ ]:
self._sa.setup_list_sweep([res_freq], [1000])


In [ ]:
plot(MR._sa.get_tracedata())

In [ ]:
MR._vna[0].sweep_continuous()
sleep(0.5)

In [ ]:
MR._open_only_readout_mixer()
MR._open_mixers()

In [ ]:
qubit_name = "VI"
DD = DispersiveDecay("%s-decay" % qubit_name,
                             MR._sample_name,
                             vna=MR._vna,
                             ro_awg=[MR._ro_awg],
                             q_awg=[MR._q_awg],
                             q_lo=MR._mw_src)

vna_parameters = {"bandwidth": 10,
                  "freq_limits": MR._res_limits[qubit_name],
                  "nop": 50,
                  "averages": 1,
                  "res_find_nop":401}

readout_delays = linspace(0, 90000, 101)
exc_frequency = MR._exact_qubit_freqs[qubit_name]
pi_pulse_duration = \
    MR._dro_results[qubit_name].get_pi_pulse_duration() * 1e3

pulse_sequence_parameters = {"awg_trigger_reaction_delay": 0,
                             "readout_duration": 12000,
                             "repetition_period": 150000,
                             "pi_pulse_duration": pi_pulse_duration}

ro_awg_params = {"calibration": MR._ro_cal}
q_awg_params = {"calibration": MR._exc_cal}

q_lo_params = {'power': MR._exc_cal.get_radiation_parameters()["lo_power"],
               'frequency': exc_frequency + MR._exc_cal.get_radiation_parameters()["if_frequency"]}

DD.set_fixed_parameters(pulse_sequence_parameters,
                vna = [vna_parameters],
                ro_awg = [ro_awg_params],
                q_awg = [q_awg_params],
                q_lo = [q_lo_params])
DD.set_swept_parameters(readout_delays)
MeasurementResult.close_figure_by_window_name("Resonator fit")
dd_result = DD.launch()

In [ ]:
dd_result.save()

In [ ]:
MR._calibrate_readout("VI")

In [ ]:
sts_result = MeasurementResult.load(MR._sample_name, "I-sts")

In [ ]:
ao = AnticrossingOracle("transmon", sts_result, plot=True, fast_res_detect=False, hints=["fqmax_below"])

In [ ]:
fres, g, period, sws, fqmax, d = ao.launch()[0]

In [ ]:
tts_result = MeasurementResult.load(MR._sample_name, "I-two-tone")

In [ ]:
reload(lib2.fulaut.SpectrumOracle)
from lib2.fulaut.SpectrumOracle import *

In [ ]:
so = SpectrumOracle('transmon', tts_result, (period, sws, fqmax, d), True)

In [ ]:
so.launch()

In [ ]:
[(1/a, b, c) for a,b,c in sorted(zip(so._coarse_brute_nop_ranking,
           so._coarse_brute_distance_ranking,
           so._coarse_brute_candidates),
                   key=itemgetter(0, 1))]

In [ ]:
so._coarse_brute_distance_ranking

In [ ]:
MR._logger.getInstance("fulaut")

In [ ]:
MR._perform_decay("IV")

In [ ]:
MR._dd_results["VI"].save()

In [ ]:
MR._perform_Ramsey_oscillations("VI")

In [ ]:
transmon_spectrum(*MR._tts_fit_params["III"])

In [ ]:
MR._tts_fit_params["III"]

In [ ]:
MR._sts_runners["I"]._sts_result.visualize()

In [ ]:
# mean([1,2,3,4,5])

In [ ]:
MR._tts_runners["III"]._tts_result.save()

In [ ]:
MR._sts

In [ ]:
MR._tts_fit_params

In [ ]:
MR._exact_qubit_freqs

In [ ]:
res = MR._sts_runners["I"]._sts_result
res

In [ ]:
res/1e6

In [ ]:
ao = AnticrossingOracle("transmon", res, True)

In [ ]:
ao.find_resonator_intersections()

In [ ]:
ao._res_points[1][0]-ao._res_points[21][0]

In [ ]:
ao.launch()

In [ ]:
data = res.get_data()

In [ ]:
freqs, data = data["frequency"], data["data"]

In [ ]:
RD = ResonatorDetector(freqs, data[5], plot=True)

In [ ]:
RD.detect()

In [ ]:
plot(freqs, abs(data[1]))

In [ ]:
from resonator_tools.circuit import notch_port

In [ ]:
NP = notch_port(freqs, data[1])

In [ ]:
NP.autofit()

In [ ]:
NP.plotall()

In [ ]:
NP.NP._fit_skewed_lorentzian(freqs, data[1])

In [ ]:
NP.fitresults